In [ ]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_diagonal_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,69824.52,-0.16,0.25,-0.27,0.05,0.03,0.02,-0.02,-0.01
3,-0.16,11408.08,0.55,-0.04,0.05,-0.00,0.00,0.00,0.00
4,0.25,0.55,2177.35,0.01,0.02,-0.00,0.00,-0.00,0.00
5,-0.27,-0.04,0.01,346.24,0.01,0.01,-0.00,-0.00,-0.00
6,0.05,0.05,0.02,0.01,86.54,0.00,-0.00,0.00,-0.00
7,0.03,-0.00,-0.00,0.01,0.00,26.35,0.00,-0.00,-0.00
8,0.02,0.00,0.00,-0.00,-0.00,0.00,10.28,-0.00,0.00
9,-0.02,0.00,-0.00,-0.00,0.00,-0.00,-0.00,1.90,-0.00
10,-0.01,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.18


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,0.00002,-0.00005,0.00002,0.00002,0.00003,-0.00005,-0.00005
3,-0.00001,1.00000,0.00011,-0.00002,0.00005,-0.00000,0.00001,0.00001,0.00004
4,0.00002,0.00011,1.00000,0.00001,0.00006,-0.00001,0.00000,-0.00003,0.00004
5,-0.00005,-0.00002,0.00001,1.00000,0.00004,0.00006,-0.00005,-0.00005,-0.00004
6,0.00002,0.00005,0.00006,0.00004,1.00000,0.00002,-0.00003,0.00001,-0.00004
7,0.00002,-0.00000,-0.00001,0.00006,0.00002,1.00000,0.00002,-0.00001,-0.00003
8,0.00003,0.00001,0.00000,-0.00005,-0.00003,0.00002,1.00000,-0.00003,0.00000
9,-0.00005,0.00001,-0.00003,-0.00005,0.00001,-0.00001,-0.00003,1.00000,-0.00005
10,-0.00005,0.00004,0.00004,-0.00004,-0.00004,-0.00003,0.00000,-0.00005,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.9699683910705045

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.41958924e+08 1.42394930e+07 5.07945605e+05 1.25146468e+04
 7.52387438e+02 6.60287141e+01 8.70130693e+00 2.25420985e-01
 1.48319401e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999772,-0.020970,-0.003883,-0.000621,-0.000154,-0.000047,-0.000018,-0.000004,-4.037778e-07
1,0.020849,0.999372,-0.028289,-0.004329,-0.001076,-0.000329,-0.000128,-0.000024,-2.030486e-06
2,0.004451,0.028065,0.999199,-0.027284,-0.006620,-0.002017,-0.000786,-0.000146,-1.312466e-05
3,0.000820,0.005000,0.026722,0.998082,-0.053193,-0.015082,-0.005862,-0.001085,-1.011829e-04
4,0.000242,0.001476,0.007757,0.051356,0.994695,-0.083607,-0.029375,-0.005335,-4.968844e-04
5,0.000089,0.000545,0.002855,0.018012,0.077260,0.985063,-0.151120,-0.022516,-2.079620e-03
6,0.000049,0.000301,0.001581,0.009961,0.040189,0.145255,0.983496,-0.099211,-8.737824e-03
7,0.000014,0.000082,0.000434,0.002726,0.010891,0.035852,0.093314,0.990488,-9.388564e-02
8,0.000003,0.000015,0.000080,0.000509,0.002032,0.006670,0.017101,0.092488,9.955423e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00022763, 0.00062768, 0.00080055, 0.00191799, 0.00530488,
       0.01493688, 0.01650376, 0.00951202, 0.00445766])